## Computing square roots

The mathematical definition of a square root tells us nothing about how to compute one algorithmically (imperative) - only about doing it symbolically in our heads through weird brain magic (declarative)

### Newton's method $\small \star$

To calculate sqrt x:

Start with a guess, then improve it by averaging $(guess + \large{\frac{x}{guess}})$

We can set a criterion for when our guess is good enough, and stop the recursion then (i.e. the boundary case)

According to the authors, declarative and imperative descriptions are very closely tied. E.g., to state that the output of a program is "correct" is a declarative statement about it.

Back in the olden days apparently people cared about abstraction enough that they were trying to design languages tha took declarative input and could produce imperative programs to fulfill it. It cannot be done in general, but you can sort-of do it sometimes. This will apparently come up again in chapter 4.

$\star$ Apparently this is a special case of it - the general form computes roots of equations


$\star \star$ Note that some exercises will be in separate notebooks

In [39]:
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))

;; where

(define (improve guess x)
  (average guess (/ x guess)))

(define (average x y)
  (/ (+ x y) 2))

(define (good-enough? guess x)
  (< (abs (- (square guess) x)) 0.0001))

;; also where

(define (square x) (* x x))

(define (abs x)
  (cond ((>= x 0) x)
        (else (- x))))

In [2]:
;; finally

(define (sqrt_ x)
  (sqrt-iter 1.0 x))

In [3]:
(displayln (sqrt_ 2))
(displayln (sqrt_ 9))

1.4142156862745097
3.000000001396984


## Exercises 1.6 - 1.8

### 1.6

In [64]:
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

(displayln (new-if (= 2 3) 0 5))

;; what if we do

(define (sqrt-iter guess x)
  (new-if (good-enough? guess x)          ;; predicate
          guess                           ;; then-clause
          (sqrt-iter (improve guess x) x) ;; else-clause
          ))

5


This one was tricky (and I had to look it up). $\text{new-if}$ will try to evaluate *all* of its arguments and will therefore keep trying to evaluate sqrt-iter, which will call itself etc. etc. It will go into an infinite loop.

### 1.7


The current $\text{sqrt_}$ function won't converge for pretty big or pretty small numbers. Fix it by adjusting the threshold to be when the change is a small fraction of the guess.

- For large numbers the difference $guess^2 - x$ will never be that small
- For small numbers the difference will be too big since the squares will be tiny (i think?)

In [40]:
;; sqrt(0.000001) = sqrt(0.0000001) - ie it doesn't converge
(> (- (sqrt_ 0.000001) (sqrt_ 0.0000001) ) 0.0001)

#f

In [41]:
;; 1.7
(define (good-enough? guess x)
  (< (abs (/ (- (square guess) x) (square guess))) 0.001))

(> (- (sqrt_ 0.000001) (sqrt_ 0.0000001) ) 0.0001)

#t

### 1.8

For cube roots, the trick is:

If $y$ is an approximation of $^3\sqrt{x}$ then a better one is given by


$$ \frac{ x / y^2 + 2y}{3} $$

Use this to get cube roots for things

I decided to make it fancy by guessing the general case for $^n \sqrt{x}$: 

$$\frac{(n - 1)y + x / y^{(n-1)} }{n}$$ 


In [65]:
(define (good-enough? guess x order)
    (< (abs (/ (- (expt guess order) x)
               (expt guess order) )) 0.00001))

(define (root-iter guess x order)
  (if (good-enough? guess x order)
      guess
      (root-iter (improve guess x order) x order)))

(define (improve guess x order)
  (/ (+ (* (- order 1) guess) (/ x (expt guess (- order 1)))) order))

(define (abs x)
  (cond ((>= x 0) x)
        (else (- x))))

In [66]:
(define (root x order)
  (root-iter 1.0 x order))

In [67]:
(println (root 8   3))
(println (root 81  4))
(println (root 96  5))
(println (root 243 6))
(println (root 243 7))

2.000004911675504
3.000000257729561
2.4914618804437842
2.498049536127603
2.1917998669575485


## Procedures as black-box abstractions

Note that we've split the recursive sqrt-iter algorithm into a number of smaller programs, each of which performs an independent computation. Because of this design, we can see util function like ***square*** like "black boxes", i.e. we don't really care about how it works, just that it does something we want - any square function is ok. *square* isn't so much a procedure as a ***procedural abstraction***. Similarly, the end user shouldn't have to care about the names of the formal parameters of the abstract procedure.

The important thing here is that parameters are local to each function, i.e. in their scope. So we can instead define the whole sqrt procedure as one big function where x is only passed around once and doesn't need to be re-defined as an argument for the functions that use it.

By extension, it's important that systems are *composable* - they reuse functions that are general to more than one task. e.g. ***good-enough?*** can be used for any arbitrary tolerance edge case in a recursive function if it's generalized appropriately.

In [2]:
(define (sqrt x)
  (define (good-enough? guess)
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess)
    (average guess (/ x  guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess)
        guess
        (sqrt-iter (improve guess))) )
  (sqrt-iter 1.0) )


(define (average x y)
  (/ 2 (+ x y)))

(define (square x)
      (* x x))


## 1.2 Procedures and the processes they generate

e.g. factorial

In [3]:
(define (factorial n)
  (if (= n 1)
      1
      (* n (factorial (- n 1)))))

In [4]:
(factorial 6)

720

The evaluation happens like so

```scheme
(factorial 6)                                 ;;
(* 6 (factorial 5))                           ;;
(* 6 (* 5 (factorial 4)))                     ;;
(* 6 (* 5 (* 4 (factorial 3))))               ;;
(* 6 (* 5 (* 4 (* 3 (factorial 2)))))         ;;
(* 6 (* 5 (* 4 (* 3 (* 2 (factorial 1))))))   ;; O(6) (i.e. O(n) ) time
(* 6 (* 5 (* 4 (* 3 (* 2 1)))))               ;;
(* 6 (* 5 (* 4 (* 3 2))))                     ;;
(* 6 (* 5 (* 4 6)))                           ;;
(* 6 (* 5 24))                                ;;
(* 6 120)                                     ;;
720

;; =============================================
;;               O(n) space
```


This is obviously quite inefficient. The expansion in the middle is due to *deferred substitution*. The process is called a ***linear recursive proces***

A different strategy is to keep a running product. Not we can still avoid it being mutable by being clever with recursion. 

In [1]:
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter product counter max-count)
    (if (> counter max-count)
        product
        (fact-iter (* counter product)
                   (+ counter 1)
                   max-count)) )

This looks better: runs in O(n) time and constant memory. It's a ***linear iterative process*** (even thouhg it's recursive)

```scheme
(factorial 6)
(fact-iter 1 1 6)
(fact-iter 1 2 6)
(fact-iter 2 3 6)
(fact-iter 6 4 6)
(fact-iter 24 5 6)
(fact-iter 120 6 6)
(fact-iter 720 7 6)
720
```

There's an interesting distinction here: Using the linear recursive version, if we Ctrl-C'd the program somewhere, we would be unable to resume the evaluation afterwards - but could pick up at any point using the iterative one. 


C-like languages need to use loops to do this.

## Exercises 1.9 -1.10

### 1.9

```scheme
(define (+ a b)
(if (= a 0) b (inc (+ (dec a) b))))

(define (+ a b)
(if (= a 0) b (+ (dec a) (inc b))))
```

The first process is **recursive** - it bloats like the naive factorial function. The second process is **iterative**

In [6]:
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1) (A x (- y 1)))) ) )

## Tree recursion

In [9]:
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ (fib (- n 1))
                 (fib (- n 2)))) ) )

If you expand the computations into a tree, you'll see that we have to compute (fib k) where k < n multiple times. This is hugely memory inefficient.

Apparently, fib(n) is the closest integer to $\phi^n / \sqrt{5} $, where $\phi = \large \frac{1 + \sqrt{5}}{2} \approx \small 1.618 $ is the golden ratio.

Generally, tree recursive processes will take exponential time in n, and linear memory in n. They can be very useful for hierarchically structured data though.

We can improve this by making it iterative like above


In [1]:
(define (fib n)
  (fib-iter 1 0 n))

(define (fib-iter a b count)
  (if (= count 0)
      b
      (fib-iter (+ a b) a (- count 1))))

## Exercises 1.11 - 1.13

### 1.11

Write this fibonacci-ish function using a recursive and an iterative process

In [ ]:
;; recursive

(define (nfib n)
  (if (< n 3)
      n
      (+ (nfib (- n 1)) (* 2 (nfib (- n 2))) (* 3 (nfib (- n 3)))) ))

In [8]:
(nfib 30) ;; it'll get hairy soon - this takes ~ 1s on my machine

61354575194

In [14]:
;; i shamefully took this solution from http://community.schemewiki.org/?sicp-ex-1.11

(define (nfib n)
    (define (nfib-iter a b c count)
      (cond ((= count 0) a)
            ((< n 3)     n)
            (else (nfib-iter (+ a (* 2 b) (* 3 c)) a b (- count 1))) ))
      (nfib-iter 2 1 0 (- n 2)) )

In [25]:
(nfib 200) ;; takes no appreciable time

429285341777505086291879477170078944250881374726914954774217395887598980875

In [1]:
;; 1.13

(define (pascal r c) 
    (if (or (= c 1) (= c r)) 
   1 
   (+ (pascal (- r 1) (- c 1)) (pascal (- r 1) c))))

In [4]:
(pascal 4 3)

3